## SQP

In [16]:
# QP

# Linesearch with penalty

# Active set strategy

# BFGS for approximating Hessian



In [38]:
# Import libraries
import sys
import numpy as np
from matplotlib import pyplot as plt
import torch as t
from torch.autograd.functional import jacobian, hessian

In [18]:
# Part 1
"""
Decision variable: x0
State variables: x1
"""

m = 2 # No. of constraints
n = 2 # No. of variables
d = n - m # No. of decision variables


def f(x):
    # Define the objective function
    f = lambda x: x[0] ** 2 + (x[1] - 3) ** 2
    # Define the constraints
    g1 = lambda x: (x[1] ** 2) - (2 * x[0])
    g2 = lambda x: (x[1] - 1) ** 2 + (5 * x[0]) - 15
    
    return f(x), g1(x), g2(x)


def Lag(x, mu):

    # Define the objective function
    f = lambda x: x[0] ** 2 + (x[1] - 3) ** 2
    # Define the constraints
    g1 = lambda x: (x[1] ** 2) - (2 * x[0])
    g2 = lambda x: (x[1] - 1) ** 2 + (5 * x[0]) - 15

    # return f(x) + mu.T @ t.tensor([[g1(x)], [g2(x)]])
    return f(x) + mu[0] * g1(x) + mu[1] * g2(x)


# Compute Jacobian
def jac(x, n=n):
    J = t.zeros((m+1, n))
    for i in range(m+1):
       J[i] =  jacobian(f, (x))[i] # 'jacobian' function in Pytorch returns a tuple of tensors. Copying each tensor slice into a new tensor for the ease of indexing.
    return J


# Compute Jacobian
def jacL(x, mu, n=n):
    J = t.zeros((1, n))
    # for i in range(n):
    J[0] =  jacobian(Lag, (x, mu))[0] # 'jacobian' function in Pytorch returns a tuple of tensors. Copying each tensor slice into a new tensor for the ease of indexing.
    return J

# Evaluate Constraints
def hFunc(x, m=m, n=n):
    H = t.zeros((m, 1))
    for i in range(m):
        H[i] =  f(x)[d + 1 + i]
    return H


def Lag1(x, mu):

    return f(x)[0] + mu @ hFunc(x)


In [10]:
def BFGS(W, x, s, dx, mu):
    Lx0 =  jacL(x - dx, mu)
    # print(Lx0.shape)
    Lx1 =  jacL(x, mu)
    # print(Lx1.shape)
    Q = dx.T @ W @ dx
    # print(dx)
    # print(dx @ (Lx1 - Lx0).T)
    # print((Lx1 - Lx0))
    L = Lx1 - Lx0
    
    if dx @ (Lx1 - Lx0).T >= 0.2 * Q:
        theta = 1
        # print(theta)
    else:
        theta = (0.8 * Q) / (Q - dx @ (Lx1 - Lx0).T)
        # print(theta)

    y = theta * (Lx1 - Lx0) + (1 - theta) * (W @ dx)
    # print(W)
    # print((s.T @ W @ s))
    # W = W + ((y.T @ y) / (y @ s.T)) - (((W @ s).reshape(-1, 1) @ (s.T @ W).reshape(1, -1)) / (s.T @ W @ s))
    W = W + ((y.T @ y) / (y @ dx.T)) - (((W @ dx).reshape(-1, 1) @ (dx.T @ W).reshape(1, -1)) / (dx.T @ W @ dx))

    # print((W))
    
    return W 

In [5]:
# Armijo Line 
def F(alpha, x, s, wj):
    dx = alpha * s
    H = hFunc(x + dx)
    F = f(x + dx)[0] + t.sum(wj.T @ t.max(t.tensor([0]), H))
    return F

def phi(alpha, x, s, wj, t0=0.5):
    phi = F(alpha, x, t.zeros((2)), wj) + t0 * alpha * dFda(alpha, x, s, wj) 

    return phi

def dFda(alpha, x, s, wj):
    J =  jac(x)
    H = hFunc(x)
    
    dgdx = J[1:, :]
    # print(dgdx.shape)
    
    dgda = dgdx @ s.reshape(-1, 1)
    # print(dgda.shape)
    dgda[(t.max(t.tensor([0]), H) <= 0)] = 0
    
    dFda = J[0, :].T @ s + t.sum(wj.T @ dgda)
    
    return dFda

def lineSearch(x, s, mu, wj0, K=25):
    alpha = 1
    i = 0
      
    wj = t.max(t.abs(mu), 0.5 * (wj0 + t.abs(mu)))
    
    # print('Fa initial: ', F(alpha, x, s, wj))
    # print('phi initial: ', phi(alpha, x, s, wj))
    # print('dFda initial: ',dFda(alpha, x, s, wj))
    

    while F(alpha, x, s, wj) + 1e-10  > phi(alpha, x, s, wj) and i < K:
        
        
        alpha = 0.5 * alpha
        print(alpha, i)
        
        F1 = F(alpha, x, s, wj)
        phi1 = phi(alpha, x, s, wj)
        dFda1 = dFda(alpha, x, s, wj)
        # print('\nFa: ', F1)
        # print('dFda: ', dFda1)
        # print('phi: ', phi1)

        i += 1
    return alpha, wj

In [6]:
def activeSet(x, s, mu, active, flag):
    A = jac(x)[1:, :]
    H = hFunc(x)
    
    constraintQP = t.round(A @ s.reshape(-1, 1) + H, decimals=3)
    
    val1 , idx1 = t.max(constraintQP, 0)
    # print('Constraint:', constraintQP)
    val2 , idx2 = t.min(mu, 0)
    
    if val2 < 0:
        # remove.append(idx2.item())
        active.pop(idx2.item())
        # print('Remove indices; ', idx2)
    
    else:
        if val1  > 0:
            # add.append(idx1.item())
            active.append(idx1.item())
            # print('Add indices; ', idx1)

        else:
            flag = True
    
    # active.append(add)
    # active.pop(remove)
    
    active = [*set(active)]
    
    return active, flag



def solveQP(x, W, mu, active):
    # W = t.eye(n)
    A = jac(x)[1:, :]
    dfx = jac(x)[0]
    h = hFunc(x)
    # mu = t.tensor([0., 0.], dtype=t.float, requires_grad=True)

    if len(active) == 0:
        X = t.linalg.solve(W, -dfx)
        s = X
        
    else:
        A = A[active]
        h = h[active]
        
        C = t.vstack((t.hstack((W, A.T)), t.hstack((A, t.zeros(A.shape[0], A.shape[0])))))
        d = - t.vstack((dfx.reshape(-1, 1), h)) # Check if this negative sign is important
        
        X = t.linalg.solve(C, d)
        s = X[:n, :]
        with t.no_grad():
            mu[active] = X[n:, :].T

    return s, mu



def QP(x, W):
    mu = t.tensor([0., 0.], dtype=t.float, requires_grad=True)
    
    flag = False
    active = []
    # i = 0
    # print('Flag:', flag)
    while flag == False:
        s, mu = solveQP(x, W, mu, active)
        active, flag = activeSet(x, s, mu, active, flag)
        # print('Counter', i)
        # i += 1
        
    return s.reshape(1, -1)[0], mu


In [13]:
# Part 4
# Initialization
def SQP(x):
    # Initialize variables
    
    
    mu = t.tensor([0., 0.], dtype=t.float, requires_grad=True)
    W = t.eye(n, dtype=t.float)
    wj = t.tensor([0., 0.], dtype=t.float)
    
    tol = 1e-3 # Error threshold
    e = t.norm(jacL(x, mu))
    
    xSol = x.detach().numpy()
    fVal = [f(x)[0].item()]
    alphaSol = [1]
    eVal = [e]

    k = 0
    while e > tol:
        
        s, muNext = QP(x, W)
        # print(f's: {s}')

        
        # Part 4.1
        # Inexact line search
        alpha, wj = lineSearch(x, s, mu, wj)
        mu = muNext
        
        # print('Mu: ', mu)
        
        # alpha = 1

        # Update the point 
        dx = alpha * s
        with t.no_grad():
            x = x + dx
        
        # Part 4.4
        # LM Solver
        W = BFGS(W, x, s, dx, muNext)

        # Part 4.5
        e = t.norm(jacL(x, muNext))
        
        
        
        # print(e)
        
        # Store important information in every iteration
        xSol = np.vstack((xSol, x.detach().numpy())) # Record x values in each iteration
        fVal.append(f(x)[0].item()) # Record f values in each iteration
        alphaSol.append(alpha) # Record alpha values in each iteration
        eVal.append(e)
        # print(f'k: {k}, x: {x}\n')
        k += 1
        print (f"Iteration: {k:<5} Alpha: {alpha:<10} x: {str(x.detach().numpy()) :<25} f(x): {fVal[k]:<25} Error: {e:<20}\n")
    return xSol, fVal, alphaSol, eVal


In [37]:
x = t.tensor([10., 0.], dtype=t.float, requires_grad=True)




val1 , idx1 = t.max(hFunc(x), 0)
if val1  > 0:
    sys.exit('Initial guess does not lie in the feasible domain!!!')

xSol, fVal, alphaSol, eVal = SQP(x)


SystemExit: Initial guess does not lie in the feasible domain!!!